In [15]:
import urllib.error


import urllib.request
import re
import rsa
import http.cookiejar  #从前的cookielib
import base64
import json
import urllib
import urllib.parse
import binascii

# 用于模拟登陆新浪微博

class launcher():

    def __init__(self,username, password):
        self.password = password
        self.username = username

    def get_prelogin_args(self):


        '''
        该函数用于模拟预登录过程,并获取服务器返回的 nonce , servertime , pub_key 等信息
        '''
        json_pattern = re.compile('\((.*)\)')
        url = 'http://login.sina.com.cn/sso/prelogin.php?entry=weibo&callback=sinaSSOController.preloginCallBack&su=&' + self.get_encrypted_name() + '&rsakt=mod&checkpin=1&client=ssologin.js(v1.4.18)'
        try:
            request = urllib.request.Request(url)
            response = urllib.request.urlopen(request)
            raw_data = response.read().decode('utf-8')
            json_data = json_pattern.search(raw_data).group(1)
            data = json.loads(json_data)
            return data
        except urllib.error as e:
            print("%d"%e.code)
            return None

    def get_encrypted_pw(self,data):
        rsa_e = 65537 #0x10001
        pw_string = str(data['servertime']) + '\t' + str(data['nonce']) + '\n' + str(self.password)
        key = rsa.PublicKey(int(data['pubkey'],16),rsa_e)
        pw_encypted = rsa.encrypt(pw_string.encode('utf-8'), key)
        self.password = ''   #清空password
        passwd = binascii.b2a_hex(pw_encypted)
        #print(passwd)
        return passwd

    def get_encrypted_name(self):
        username_urllike   = urllib.request.quote(self.username)
        username_encrypted = base64.b64encode(bytes(username_urllike,encoding='utf-8'))
        return username_encrypted.decode('utf-8')

    def enableCookies(self):
            #建立一个cookies 容器
            cookie_container = http.cookiejar.CookieJar()
            #将一个cookies容器和一个HTTP的cookie的处理器绑定
            cookie_support = urllib.request.HTTPCookieProcessor(cookie_container)
            #创建一个opener,设置一个handler用于处理http的url打开
            opener = urllib.request.build_opener(cookie_support, urllib.request.HTTPHandler)
            #安装opener，此后调用urlopen()时会使用安装过的opener对象
            urllib.request.install_opener(opener)


    def build_post_data(self,raw):
        post_data = {
            "entry":"weibo",
            "gateway":"1",
            "from":"",
            "savestate":"7",
            "useticket":"1",
            "pagerefer":"http://passport.weibo.com/visitor/visitor?entry=miniblog&a=enter&url=http%3A%2F%2Fweibo.com%2F&domain=.weibo.com&ua=php-sso_sdk_client-0.6.14",
            "vsnf":"1",
            "su":self.get_encrypted_name(),
            "service":"miniblog",
            "servertime":raw['servertime'],
            "nonce":raw['nonce'],
            "pwencode":"rsa2",
            "rsakv":raw['rsakv'],
            "sp":self.get_encrypted_pw(raw),
            "sr":"1280*800",
            "encoding":"UTF-8",
            "prelt":"77",
            "url":"http://weibo.com/ajaxlogin.php?framelogin=1&callback=parent.sinaSSOController.feedBackUrlCallBack",
            "returntype":"META"
        }
        data = urllib.parse.urlencode(post_data).encode('utf-8')
        return data


    def login(self):
        url = 'http://login.sina.com.cn/sso/login.php?client=ssologin.js(v1.4.18)'
        self.enableCookies()
        data = self.get_prelogin_args()
        post_data = self.build_post_data(data)
        headers = {
            "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.87 Safari/537.36"
        }
        try:
            request = urllib.request.Request(url=url,data=post_data,headers=headers)
            response = urllib.request.urlopen(request)
            html = response.read().decode('GBK')
            #print(html)
        except urllib.error as e:
            print(e.code)

        p = re.compile('location\.replace\(\'(.*?)\'\)')
        p2 = re.compile(r'"userdomain":"(.*?)"')

        try:
            login_url = p.search(html).group(1)
            #print(login_url)
            request = urllib.request.Request(login_url)
            response = urllib.request.urlopen(request)
            page = response.read().decode('utf-8')
            #print(page)
            login_url = 'http://weibo.com/' + p2.search(page).group(1)
            request = urllib.request.Request(login_url)
            response = urllib.request.urlopen(request)
            final = response.read().decode('utf-8')
            #print(final)

            print("Login success!")
        except:
            print('Login error!')
            return 0

In [16]:
a=launcher('elivthrar@gmail.com','j31415926')

In [17]:
a.login()

Login success!


In [18]:
page=urllib.request.urlopen('http://vdisk.weibo.com/')

In [19]:
page.read().decode('utf-8')

In [30]:
url='http://vdisk.weibo.com/share/linkfolderSave'
headers = {
    'Host':' vdisk.weibo.com',
'Connection':' keep-alive',
'Content-Length':' 153',
'Origin':' http://vdisk.weibo.com',
'User-Agent':' Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.87 Safari/537.36',
'Content-Type':' application/x-www-form-urlencoded; charset=UTF-8',
'Accept':' application/json, text/javascript, */*; q=0.01',
'x-response-version':' 2',
'X-Requested-With':' XMLHttpRequest',
'Referer':' http://vdisk.weibo.com/lc/f7C4p3rotMNLayOvY',
        }
post_data={
    'root':'basic',
    'to_path': '/backup/妖魔鬼怪 神话传说',
    'access_code':'I651',
    'from_copy_ref':'f7C4p3rotMNLayOvY',
}
data = urllib.parse.urlencode(post_data).encode('utf-8')
print(data)
request = urllib.request.Request(url=url,data=data,headers=headers)
response = urllib.request.urlopen(request)
html = response.read().decode('GBK')
print(html)

In [40]:
re.findall('modified',html)

In [61]:
import csv
with open('share.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='	', quotechar='|')
    for row in spamreader:
        print(row[2].strip())

Y027
NMU4
X3SP
WJK8
MQ59
TT32
GFRH
HN5D
HOGA
WK48
Y30C
XP3Z
YQEN
T3I6
K25L
JY2N
GZB8
F485
W0H1
TAL5
A63R
X4SQ
D3V8
KFVU
U5UF
X1EW
G1WH
J4QW
T609
X9D1
EB39
T22J
IPV6
DU9C
F41U
N866
ZOPH
P356
Q1S8
FDSA
YRJO
V43K
YRJO
X1K0
P23S
CZR0
P71M
J2G5
EDMP
EN39
LVK9
GRPO
KA8T
G262
J599
Q074
G78Z
UOK3
ZX42
D7Y1
K6FB
QL51
GL4O
XPR1
G921
AZU4
ZVP5
MQ17
B926
K6JF
Q3J4
OHDU
P5H6
IFBI
K14K
U73Q
X5U8
EXT6
LO4H
X0I1
A81Z
FRWR
GFA4
AL01
KPGL
U1C6
K53H
S3Y8
F754
NKK4
Z8U3
ZD80
I763
BG97
ZDY1
X72X
G163
YF68
P153
MQ2R
Y579
UP6K
I460
JON4
A97M
H7X8
YZXZ
ED5F
D3Y9
PVWR
J853
B96D
B011
NH21
A794
MGQ7
ZEB8
OYV0
ZHK8
A255
H49A
TBIM
X2SJ
BF6B
QKC4
S034
RWYN
W3N5
YM3A
X0AH
A6U9
I651
OY54
Z5NI
P837
Q116
XA9N
SC5Q
ES1N
H7T9
L24G
W1F5
Y3Y6
S86S
ND8R
O3H7
NSYQ
ZLIC
J345
NYY8
P0VX
S750
B6OL
UPW1
WW3S
NDG0
OEAF
D27D
XWCI
XWCI
TAVF
AR4D
DQOW
YCB8
TY27
V8I9
Z336
DZ61
VN8Q
TG70
ZOGK
DAP3
N3N6
QVM9
XX2M
RW3F
P26M
I4U6
K999
YQFD
CX68
F837
R95B
QDEE
W116
PWQQ
BEU9
DRG9
LLJR
GI8V
B957
WH0B
G95T
RH7J
B7UZ
S7O5
HBVP
C14U
GGKQ
F181
